# Natural Language Processing Using RNNs and Attention

#### Generating Shakespeare using a character RNN

In [16]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

In [17]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [18]:
#now encode every character as an integer

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [19]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [20]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [21]:
max_id = len(tokenizer.word_index)

In [22]:
#encode the full text so each character is represented by its ID

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [23]:
#splitting the data set needs to be done carefully, since it is sequential

dataset_size = encoded.shape[0]
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [24]:
#we need to use the data sets window method to convert the long string of characters to smaller readable pieces.

n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [25]:
#flatten the data set for training

dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [26]:
#now we can shuffle the windows 

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [27]:
#we only havee 39 distinct characters so lets encode them as one hot vectors

dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1)

In [28]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [29]:
#now the data has been prepared and we can move on to creating the model.

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None,max_id],
                    dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                    dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

In [30]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
history = model.fit(dataset, epochs=20)

Epoch 1/20
31368/31368 [==============================] - 11527s 367ms/step - loss: 1.6202
Epoch 2/20
31368/31368 [==============================] - 2454s 78ms/step - loss: 1.5383
Epoch 3/20
31368/31368 [==============================] - 2406s 77ms/step - loss: 1.5175
Epoch 4/20
31368/31368 [==============================] - 2402s 77ms/step - loss: 1.5066
Epoch 5/20
31368/31368 [==============================] - 2413s 77ms/step - loss: 1.4988
Epoch 6/20
31368/31368 [==============================] - 2417s 77ms/step - loss: 1.4936
Epoch 7/20
31368/31368 [==============================] - 35558s 1s/step - loss: 1.4896
Epoch 8/20
31368/31368 [==============================] - 2494s 79ms/step - loss: 1.4863
Epoch 9/20
31368/31368 [==============================] - 2570s 82ms/step - loss: 1.4838
Epoch 10/20
31368/31368 [==============================] - 278355s 9s/step - loss: 1.4810
Epoch 11/20
31368/31368 [==============================] - 2622s 84ms/step - loss: 1.4789
Epoch 12/20
31368/

In [31]:
#This model predicts the next character written by Shakespeare. Preprocessing is required to feed text.

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [33]:
#test it out with a simple sentence

X_new = preprocess(['How are yo'])
Y_pred = np.argmax(model.predict(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

1/1 [==============================] - 1s 882ms/step


'u'

In [40]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    Y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(Y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [51]:
#with these functions we can now print out some text.

print(complete_text('Who are you', temperature=1))

1/1 [==============================] - 0s 94ms/step
Who are you profumed 
we laysing more sisters to his reason.

katharina:
well i never kath rival him my wife, i


In [ ]:
#Hardly King Lear, but we can see some characteristics of Shakespearean prose!